<a href="https://colab.research.google.com/github/ProfessorDong/Deep-Learning-Course-Examples/blob/master/RNN_Examples/CharacterGeneratorRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Natural Language Processing (Character Generation) with Recurrent Neural Network


We will generate charater using a RNN.  We will work with a dataset of Shakespeare's writing.  Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

- The model is character-based.
- The model is trained on small batches of text (100 characters each), and is able to generate a longer sequence of text with coherent structure.

*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*


In [1]:
# %tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

###Download the Shakespeare Dataset

Here, we use an extract from a Shadespheare play for training.  We can use our own text paragraph data and use it for network training.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# If you load your own data file, use the following lines:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

###Read Contents of File
Let's look at the contents of the file.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


###Encode
Encode each unique character as a different integer.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# Chect how a part of the text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Make a function that can convert the numeric values back to text.


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Create Training Examples

Given a character, or a sequence of characters, what is the most probable next character? This is the task we are training the model to perform.

Our task is to feed the model a sequence and have it return the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

In [9]:
# Create a stream of characters from our text data
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Use the batch method to turn this stream of characters into batches of desired length
seq_length = 100  # length of sequence for a training example
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
# Use these sequences of length 101 and split them into input and output.
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(3):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


EXAMPLE

INPUT
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us k

OUTPUT
ow Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki


Make training batches.

In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Build the Model
We use an embedding layer, a LSTM layer, and one dense layer that contains a node for each unique character in our training data.  The dense layer will give us a probability distribution over all nodes.

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    # tf.keras.layers.LSTM(rnn_units,
    #                     return_sequences=True,
    #                     stateful=True,
    #                     recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),                        
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


###Create a Loss Function
Our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

We will create the loss function.

Before we do that, let's have a look at a sample input and the output from the untrained model. 

In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-7.07990117e-03 -4.76442743e-03 -2.04344019e-02 ... -4.96687228e-03
    1.56598259e-02  8.93331319e-03]
  [-1.17754936e-02 -1.37073756e-03 -8.82906187e-03 ... -1.44832954e-03
    6.20673236e-05 -3.96777084e-03]
  [-3.67854722e-03  4.46888525e-03  1.15210842e-02 ... -5.93594555e-03
   -6.00861106e-03 -5.89860929e-03]
  ...
  [-1.57112745e-03  1.48910917e-02 -8.65585543e-03 ... -1.75735988e-02
    1.70896538e-02  1.64080169e-02]
  [-4.44705319e-03  9.01354291e-03 -7.57682975e-03 ... -1.46965142e-02
    1.28477160e-02  4.53207782e-03]
  [-7.04125827e-03  7.44407531e-04 -2.34465152e-02 ... -1.11233266e-02
    2.49605365e-02  1.31133730e-02]]

 [[-1.52290016e-02  8.29744153e-03 -1.04310848e-02 ...  7.96104316e-03
   -5.33781294e-03 -2.71290401e-03]
  [-1.19375112e-02 -1.42400553e-02  1.27034057e-02 ...  1.68188810e-02
   -5.93215320e-03  5.27501153e-03]
  [-1.17188366e-03 -9.30668600e-03  2.66785920e-03 ... -2.15611584e-03
   -2.85003241e-03 -6.33151038e-04]
  ...
  [ 9.824

In [15]:
# Let's examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-7.0799012e-03 -4.7644274e-03 -2.0434402e-02 ... -4.9668723e-03
   1.5659826e-02  8.9333132e-03]
 [-1.1775494e-02 -1.3707376e-03 -8.8290619e-03 ... -1.4483295e-03
   6.2067324e-05 -3.9677708e-03]
 [-3.6785472e-03  4.4688853e-03  1.1521084e-02 ... -5.9359455e-03
  -6.0086111e-03 -5.8986093e-03]
 ...
 [-1.5711274e-03  1.4891092e-02 -8.6558554e-03 ... -1.7573599e-02
   1.7089654e-02  1.6408017e-02]
 [-4.4470532e-03  9.0135429e-03 -7.5768298e-03 ... -1.4696514e-02
   1.2847716e-02  4.5320778e-03]
 [-7.0412583e-03  7.4440753e-04 -2.3446515e-02 ... -1.1123327e-02
   2.4960537e-02  1.3113373e-02]], shape=(100, 65), dtype=float32)


In [16]:
# and finally we look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-0.0070799  -0.00476443 -0.0204344   0.00837173 -0.0002283  -0.00485945
  0.00414648  0.02027825  0.0084383  -0.00044633 -0.02117979  0.01560126
 -0.00246244 -0.00374136 -0.00888249  0.00791233 -0.0116054   0.00536097
  0.01006107 -0.01547956  0.00301468 -0.00834186 -0.00110226  0.00986686
  0.00527686 -0.0103611   0.01449836  0.00940825 -0.00968575 -0.00112415
 -0.00453141  0.00138459 -0.00189479  0.00629974 -0.00082139  0.0048298
 -0.0149695   0.00499609  0.00195562 -0.00816927  0.00290527  0.00316919
 -0.01413079  0.00569141  0.00802276 -0.00109389 -0.00380627 -0.00225127
 -0.00204624  0.00051786  0.00973385  0.00116415 -0.00836793 -0.01241091
  0.0071595   0.00847059 -0.01256029 -0.00516613 -0.00386131  0.01291503
 -0.00451989 -0.00218984 -0.00496687  0.01565983  0.00893331], shape=(65,), dtype=float32)


In [17]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"IcH-mO:ZZVt;tXINELJK,3XxW?f'kWEy:PwFykA.xd;Pq,SkK\nbM,DglTcwBFmG;cHsx;U'xGNfJwrzfndKMEm\nKYed$ZMu:kfwm"

So, we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were. 

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compile the Model
We can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next. 


In [19]:
model.compile(optimizer='adam', loss=loss)

###Create Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Train the Model
**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**

In [21]:
history = model.fit(data, epochs=30, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 19s 71ms/step - loss: 2.6596
Epoch 2/30
172/172 [==============================] - 12s 59ms/step - loss: 1.9630
Epoch 3/30
172/172 [==============================] - 12s 60ms/step - loss: 1.6910
Epoch 4/30
172/172 [==============================] - 12s 61ms/step - loss: 1.5417
Epoch 5/30
172/172 [==============================] - 12s 62ms/step - loss: 1.4517
Epoch 6/30
172/172 [==============================] - 12s 62ms/step - loss: 1.3934
Epoch 7/30
172/172 [==============================] - 12s 61ms/step - loss: 1.3473
Epoch 8/30
172/172 [==============================] - 12s 61ms/step - loss: 1.3090
Epoch 9/30
172/172 [==============================] - 12s 60ms/step - loss: 1.2747
Epoch 10/30
172/172 [==============================] - 12s 60ms/step - loss: 1.2421
Epoch 11/30
172/172 [==============================] - 12s 60ms/step - loss: 1.2099
Epoch 12/30
172/172 [==============================] - 12s 62ms/step - loss: 1.1780
E

###Load the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [22]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model finishes training, we can find the **lastest checkpoint** that stores the models weights using the following line.

In [23]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [24]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

###Generate Text
Generate some text using any starting string we choose.

Generating text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.  Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

![alt text](https://www.tensorflow.org/text/tutorials/images/text_generation_sampling.png)


In [25]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [26]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: The sky is blue, so is my heart
The sky is blue, so is my heart;
With sighs and tr be admitting her,
As on a hail, her accusations, yea
O close fit on the sanctibenies.

Second Servingman:
Now is't without the fire, encounter, but
a rule my teeth-bow in the world's voices,
I will be ta'en fornt but bold ingee
Of your blood is still our prayer
gone: I'll not become my heart
To make corn, laments, and that vase women,
And where the king's palea; whose soul is fall, for curbat
What he begain and altering in his
die and my profit steeds most filthrow
To tell him sinfeir than thou, banished!

HASTINGS:
Then pardon is nothing; and they are that modesty.

QUEEN ELIZABETH:
Plain and quivers. For us, to wear the neck
Of that this the ear in Such a villain,
I would where you will rid's cornz, must high Henry the time-forfeit of my master.

PETRUCHIO:
Nay, I wil


##Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
3. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.